# &#x1F4D1; **NLI - Bert Version**

用Bert来进行QNLI训练集的蕴含分析



## &#x2728;任务描述

- 中文抽取式问答
    - 输入：问题 + 回答
    - 输出：回答是否对得上此问题，即有没有答非所问

- 目标
    - 学习如何使用transformers在下游任务上微调预训练模型

## 准备数据集
从本地上传TA提供的两个tsv文件

In [1]:
from google.colab import files

# 选择并上传文件
uploaded = files.upload()

Saving train_40.tsv to train_40 (1).tsv


In [2]:
from google.colab import files

# 选择并上传文件
uploaded = files.upload()

Saving dev_40.tsv to dev_40 (1).tsv


装个库，就可以直接调用Bert

In [3]:
! pip install transformers==4.41.2

直接开始运行

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import time

class QNLIDataset(torch.utils.data.Dataset):
    def __init__(self, file_path, tokenizer, max_length):
        self.sentences = []
        self.labels = []
        self.max_length = max_length

        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()[1:]  # Skip header

        for line in lines:
            items = line.strip().split('\t')
            premise = items[1]
            hypothesis = items[2]
            # this sign
            sentence = "[CLS] " + premise + " [SEP] " + hypothesis + " [SEP]"
            encoded_dict = tokenizer.encode_plus(
                sentence,
                add_special_tokens=True,
                max_length=max_length,
                pad_to_max_length=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
            self.sentences.append((encoded_dict['input_ids'], encoded_dict['attention_mask']))
            self.labels.append(int(items[3] == 'entailment'))

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.sentences[idx], self.labels[idx]

class BERTModel(nn.Module):
    def __init__(self, output_size, pretrained_model_name='bert-base-uncased'):
        super(BERTModel, self).__init__()
        self.bert = BertModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        dropped_out = self.dropout(pooled_output)
        return self.fc(dropped_out)

def train(model, data_loader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch in data_loader:
        (input_ids, attention_mask), labels = batch
        input_ids = input_ids.squeeze(1).to(device)
        attention_mask = attention_mask.squeeze(1).to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item() * input_ids.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def evaluate(model, data_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            (input_ids, attention_mask), labels = batch
            input_ids = input_ids.squeeze(1).to(device)
            attention_mask = attention_mask.squeeze(1).to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * input_ids.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

if __name__ == "__main__":
    pretrained_model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)

    train_file = '/content/train_40.tsv'
    valid_file = '/content/dev_40.tsv'

    max_length = 50
    batch_size = 16

    train_dataset = QNLIDataset(train_file, tokenizer, max_length)
    valid_dataset = QNLIDataset(valid_file, tokenizer, max_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    print('Number of training samples:', len(train_dataset))
    print('Number of validation samples:', len(valid_dataset))

    output_size = 2
    num_epochs = 3
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BERTModel(output_size, pretrained_model_name).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
    total_steps = len(train_loader) * num_epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

    loss = []
    accuracy = []

    for epoch in range(num_epochs):
        start_time = time.time()
        train_loss, train_acc = train(model, train_loader, criterion, optimizer, scheduler, device)
        valid_loss, valid_acc = evaluate(model, valid_loader, criterion, device)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Valid Loss: {valid_loss:.4f}, Valid Acc: {valid_acc:.4f}')
        end_time = time.time()
        loss.append(valid_loss)
        accuracy.append(valid_acc)
        elapsed_time = end_time - start_time
        print('Elapsed Time:', elapsed_time, 's')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2699: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Number of training samples: 74036
Number of validation samples: 3736


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch [1/3], Train Loss: 0.4001, Train Acc: 0.8191, Valid Loss: 0.2778, Valid Acc: 0.8825
Elapsed Time: 739.0560297966003 s
Epoch [2/3], Train Loss: 0.2251, Train Acc: 0.9117, Valid Loss: 0.2961, Valid Acc: 0.8804
Elapsed Time: 748.6208920478821 s
Epoch [3/3], Train Loss: 0.1181, Train Acc: 0.9585, Valid Loss: 0.3267, Valid Acc: 0.8919
Elapsed Time: 748.5845053195953 s
